# 浅层神经网络的简单实现

输入层维度为$n_x$，隐藏层维度为$n_h$，输出层维度为$n_y$：
$n_x = 4$, $n_h = 8$, $n_y = 1$

隐藏层激活函数为$tanh$，输出层激活函数为$sigmoid$。

损失函数为交叉熵损失函数：
$$  L = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log{\hat{y}^{(i)}} + (1 - y^{(i)})\log{(1 - \hat{y}^{(i)})} $$

其中，$m$为样本数，$y^{(i)}$为第$i$个样本的目标值，$\hat{y}^{(i)}$为第$i$个样本的输出值。

前向传播：

<img alt="前向传播" height="1500" src="./images/前向传播.png" width="2000"/> 


反向传播：

<img alt="反向传播" height="1000" src="./images/反向传播.png" width="2000"/>



In [13]:
import numpy as np
import pandas as pd
from tqdm import *

## 1. 数据预处理

输入数据的维度为$(4, 80)$，其中$4$为特征数，$80$为样本数。输出数据的维度为$(1, 80)$，其中$80$为样本数。

In [4]:
# 读取数据
origin_data = pd.read_csv('../../Datasets/exercise_data/Iris.csv', header=0, nrows=100, index_col=0)
species = origin_data["Species"].unique()
origin_data["Species"] = origin_data["Species"].map({species[0]: 0, species[1]: 1})

# 打乱数据
shuffle_data = origin_data.sample(frac=1)

# 划分训练集和测试集
train_X = shuffle_data.iloc[:80, :-1].values.T
train_Y = shuffle_data.iloc[:80, -1].values.reshape(1, -1)

test_X = shuffle_data.iloc[80:, :-1].values.T
test_Y = shuffle_data.iloc[80:, -1].values.reshape(1, -1)

display(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(4, 80)

(1, 80)

(4, 20)

(1, 20)

## 2. 搭建神经网络

### 2.1 定义神经网络结构

In [6]:
def layer_sizes(X, Y):
    """
    定义神经网络结构
    :param X: 输入数据，维度为(特征数, 样本数)
    :param Y: 输出数据，维度为(1, 样本数)
    :return: 输入层维度、隐藏层维度、输出层维度
    """
    # 输入层维度
    n_x = X.shape[0]

    # 隐藏层维度
    n_h = 8

    # 输出层维度
    n_y = Y.shape[0]
    return n_x, n_h, n_y

### 2.2 初始化模型参数

In [8]:
def initialize_parameters(n_x, n_h, n_y):
    """
    初始化模型参数
    :param n_x: 输入层维度
    :param n_h: 隐藏层维度
    :param n_y: 输出层维度
    :return: 模型参数
    """
    # 随机种子
    np.random.seed(2)

    # 随机初始化隐藏层参数
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))

    # 随机初始化输出层参数
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))

    # 使用断言确保数据格式正确
    assert W1.shape == (n_h, n_x), "W1的维度应该是({}, {})".format(n_h, n_x)
    assert b1.shape == (n_h, 1), "b1的维度应该是({}, 1)".format(n_h)

    assert W2.shape == (n_y, n_h), "W2的维度应该是({}, {})".format(n_y, n_h)
    assert b2.shape == (n_y, 1), "b2的维度应该是({}, 1)".format(n_y)

    # 以字典形式返回参数
    parameters = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }
    return parameters

### 2.3 sigmoid 函数以及导数

In [7]:
def sigmoid(Z):
    """
    sigmoid 函数
    :param Z: 输入数据
    :return: sigmoid 函数值
    """
    A = 1 / (1 + np.exp(-Z))
    return A


def sigmoid_derivative(Z):
    """
    sigmoid 函数导数
    :param Z: 输入数据
    :return: sigmoid 函数导数值
    """
    A = sigmoid(Z)
    return A * (1 - A)

### 2.4 前向传播

In [12]:
def forward_propogation(X, parameters):
    """
    前向传播
    :param X: 输入数据，维度为(特征数, 样本数)
    :param parameters: 模型参数
    :return: 隐藏层激活值、输出层激活值
    """
    # 获取模型参数
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    # 前向传播计算隐藏层激活值
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)

    # 前向传播计算输出层激活值
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)

    # 使用断言确保数据格式正确
    assert A2.shape == (1, X.shape[1]), "A2的维度应该是(1, {})".format(X.shape[1])

    # 以字典形式返回隐藏层激活值和输出层激活值
    cache = {
        "Z1": Z1,
        "A1": A1,
        "Z2": Z2,
        "A2": A2
    }
    return cache

### 2.5 计算损失函数

In [11]:
def loss_function(A2, Y):
    """
    计算损失函数
    :param A2: 输出层激活值
    :param Y: 输出数据
    :return: 损失函数值
    """
    # 求每个样本输出值与目标值的交叉熵
    cross_entropy = -np.multiply(np.log(A2), Y) - np.multiply(np.log(1 - A2), (1 - Y))

    # 求所有样本交叉熵的均值
    loss = np.mean(cross_entropy)

    # 将得到的 1x1 损失值矩阵转换为标量
    loss = np.squeeze(loss)

    return loss

### 2.6 反向传播

In [18]:
def back_propogation(parameters, cache, X, Y):
    """
    反向传播
    :param parameters: 模型参数
    :param cache: 前向传播得到的隐藏层激活值和输出层激活值
    :param X: 输入数据
    :param Y: 输出数据
    :return: 模型参数的梯度
    """
    # 获取样本数
    m = X.shape[1]

    # 获取模型参数
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]

    # 获取前向传播得到的隐藏层激活值和输出层激活值
    A1 = cache["A1"]
    A2 = cache["A2"]

    # 计算输出层激活值相对于损失函数的导数
    dZ2 = A2 - Y

    # 计算输出层参数相对于损失函数的导数
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m

    # 计算隐藏层激活值相对于损失函数的导数
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))

    # 计算隐藏层参数相对于损失函数的导数
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m

    # 使用断言确保数据格式正确
    assert dW2.shape == W2.shape, "dW2的维度应该是{}".format(W2.shape)
    assert db2.shape == b2.shape, "db2的维度应该是{}".format(b2.shape)
    assert dW1.shape == W1.shape, "dW1的维度应该是{}".format(W1.shape)
    assert db1.shape == b1.shape, "db1的维度应该是{}".format(b1.shape)

    # 以字典形式返回模型参数的梯度
    grads = {
        "dW2": dW2,
        "db2": db2,
        "dW1": dW1,
        "db1": db1
    }

    return grads

### 2.7 更新模型参数

In [9]:
def update_parameters(parameters, grads, learning_rate=0.005):
    """
    更新模型参数
    :param parameters: 模型参数
    :param grads: 模型参数的梯度
    :param learning_rate: 学习率
    :return: 更新后的模型参数
    """
    # 获取模型参数
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]

    # 获取模型参数的梯度
    dW1 = grads["dW1"]
    dW2 = grads["dW2"]
    db1 = grads["db1"]
    db2 = grads["db2"]

    # 更新模型参数
    W1 -= learning_rate * dW1
    W2 -= learning_rate * dW2
    b1 -= learning_rate * db1
    b2 -= learning_rate * db2

    # 使用断言确保数据格式正确
    assert W1.shape == dW1.shape, "W1的维度应该是{}".format(dW1.shape)
    assert W2.shape == dW2.shape, "W2的维度应该是{}".format(dW2.shape)
    assert b1.shape == db1.shape, "b1的维度应该是{}".format(db1.shape)
    assert b2.shape == db2.shape, "b2的维度应该是{}".format(db2.shape)

    # 以字典形式返回更新后的模型参数
    parameters = {
        "W1": W1,
        "W2": W2,
        "b1": b1,
        "b2": b2
    }
    return parameters

### 2.8 搭建模型训练逻辑

In [15]:
def nn_model(train_X, train_Y, iterarion_num=2000, learning_rate=0.005, TOL=1e-6, print_loss=False):
    """
    模型训练
    :param train_X: 训练集输入数据
    :param train_Y: 训练集输出数据
    :param iterarion_num: 迭代次数
    :param learning_rate: 学习率
    :param TOL: 两次损失函数差值阈值
    :param print_loss: 是否打印损失函数值
    :return: 模型参数
    """
    # 获取输入层维度、隐藏层维度、输出层维度
    n_x, n_h, n_y = layer_sizes(train_X, train_Y)

    # 初始化模型参数
    parameters = initialize_parameters(n_x, n_h, n_y)

    # 迭代训练
    for i in trange(1, iterarion_num + 1):
        # 前向传播
        cache = forward_propogation(train_X, parameters)

        # 计算损失函数
        loss = loss_function(cache["A2"], train_Y)

        # 反向传播
        grads = back_propogation(parameters, cache, train_X, train_Y)

        # 更新模型参数
        parameters = update_parameters(parameters, grads, learning_rate)

        # 打印损失函数值
        if print_loss and i % 100 == 0:
            print("第{}次迭代，损失函数值为：{}".format(i, loss))

        # 判断损失函数是否小于阈值
        if loss < TOL:
            break

    return parameters

### 2.9 模型预测函数

In [27]:
def predict(parameters, test_X):
    """
    模型预测函数
    :param parameters: 模型参数
    :param test_X: 测试集输入数据
    :return: 预测结果
    """
    # 前向传播
    cache = forward_propogation(test_X, parameters)

    # 获取输出层激活值
    A2 = cache["A2"]

    # 将输出层激活值大于0.5的置为1，小于0.5的置为0
    predictions = np.where(A2 > 0.5, 1, 0)

    return predictions


def calculate_accuracy(predictions, test_Y):
    """
    计算准确率
    :param predictions: 预测结果
    :param test_Y: 测试集输出数据
    :return: 准确率
    """
    # 获取测试集样本数
    m = test_Y.shape[1]

    # 计算预测正确的样本数
    correct_num = np.sum(predictions == test_Y)

    # 计算准确率
    accuracy = correct_num / m
    accuracy = np.squeeze(accuracy)
    return accuracy

## 3. 模型训练与评估

In [38]:
# 模型训练
parameters = nn_model(train_X, train_Y, iterarion_num=2000, learning_rate=0.005, TOL=1e-6, print_loss=True)

# 模型预测
predictions = predict(parameters, test_X)

# 计算准确率
accuracy = calculate_accuracy(predictions, test_Y)

print("准确率为：{:.2f}%\n".format(accuracy * 100))

for param in parameters:
    print("{} = {}\n".format(param, parameters[param]))

100%|██████████| 2000/2000 [00:00<00:00, 13626.41it/s]

第100次迭代，损失函数值为：0.6920308003790124
第200次迭代，损失函数值为：0.6908368971711158
第300次迭代，损失函数值为：0.6887440581265976
第400次迭代，损失函数值为：0.6848505160509772
第500次迭代，损失函数值为：0.6776038298050123
第600次迭代，损失函数值为：0.6644088910872272
第700次迭代，损失函数值为：0.6412401921757865
第800次迭代，损失函数值为：0.6031708750296307
第900次迭代，损失函数值为：0.5468883280466772
第1000次迭代，损失函数值为：0.4746543182744588
第1100次迭代，损失函数值为：0.3955720560115348
第1200次迭代，损失函数值为：0.3207991611307584
第1300次迭代，损失函数值为：0.2575295153308824
第1400次迭代，损失函数值为：0.20751913427520557
第1500次迭代，损失函数值为：0.16923644957802975
第1600次迭代，损失函数值为：0.14016398805132757
第1700次迭代，损失函数值为：0.11795986375724161
第1800次迭代，损失函数值为：0.10078974079650097
第1900次迭代，损失函数值为：0.08731061417989991
第2000次迭代，损失函数值为：0.07656268154212359
准确率为：100.00%

W1 = [[ 0.11342406  0.43252048 -0.66576254 -0.2641163 ]
 [-0.00943954 -0.03960593  0.07908106  0.017851  ]
 [-0.08257304 -0.31539016  0.479831    0.22740062]
 [-0.08025473 -0.33809228  0.50601636  0.21038116]
 [ 0.0194829   0.12926242 -0.20268628 -0.08302727]
 [ 0.04566165  0.20737098 -0